In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

### 데이터 작업하기

In [2]:
training_data = datasets.FashionMNIST(root="data",
                                      train=True,
                                      download=True,
                                      transform=ToTensor())

test_data = datasets.FashionMNIST(root="data", 
                                  train=False, 
                                  download=True, 
                                  transform=ToTensor())

In [3]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for x, y in test_dataloader:
    print("shape of x [N, C, H, W]: ", x.shape)
    print("shape of y: ", y.shape, y.dtype)
    break

shape of x [N, C, H, W]:  torch.Size([64, 1, 28, 28])
shape of y:  torch.Size([64]) torch.int64


### 모델 만들기

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cpu device


In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(nn.Linear(28*28, 512),
                                               nn.ReLU(),
                                               nn.Linear(512, 512),
                                               nn.ReLU(),
                                               nn.Linear(512, 10),
                                               nn.ReLU())
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


### 모델 매개변수 최적화하기

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [7]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    
    for batch, (x, y) in enumerate(dataloader):
        x, y = x.to(device), y.to(device)
        pred = model(x)
        
        loss = loss_fn(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(x)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [8]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    
    model.eval()
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \nAccuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [9]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n----------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
----------------------------
loss: 2.305527 [    0/60000]
loss: 2.300767 [ 6400/60000]
loss: 2.299908 [12800/60000]
loss: 2.294927 [19200/60000]
loss: 2.280495 [25600/60000]
loss: 2.279396 [32000/60000]
loss: 2.282068 [38400/60000]
loss: 2.277322 [44800/60000]
loss: 2.262551 [51200/60000]
loss: 2.252201 [57600/60000]
Test Error: 
Accuracy: 26.9%, Avg loss: 2.259496 

Epoch 2
----------------------------
loss: 2.275875 [    0/60000]
loss: 2.272687 [ 6400/60000]
loss: 2.269413 [12800/60000]
loss: 2.258218 [19200/60000]
loss: 2.229206 [25600/60000]
loss: 2.225868 [32000/60000]
loss: 2.243240 [38400/60000]
loss: 2.224674 [44800/60000]
loss: 2.191992 [51200/60000]
loss: 2.195480 [57600/60000]
Test Error: 
Accuracy: 27.2%, Avg loss: 2.196062 

Epoch 3
----------------------------
loss: 2.214830 [    0/60000]
loss: 2.211473 [ 6400/60000]
loss: 2.188206 [12800/60000]
loss: 2.191197 [19200/60000]
loss: 2.156707 [25600/60000]
loss: 2.140849 [32000/60000]
loss: 2.174319 [38400/60000]
loss

In [11]:
torch.save(model.state_dict(), "model.pth")
print("saved torch model state to model.pth")

saved torch model state to model.pth


### 모델 불러오기

In [12]:
new_model = NeuralNetwork()
new_model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [13]:
classes = ["T-shirt/top",
           "Trouser",
           "Pullover",
           "Dress",
           "Coat",
           "Sandal",
           "Shirt",
           "Sneaker",
           "Bag",
           "Ankle boot"
]

In [15]:
model.eval()
x, y = test_data[0][0], test_data[0][1]

with torch.no_grad():
    pred = new_model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Sneaker", Actual: "Ankle boot"
